# Ultralytics Classifier

Set up runtime to use GPU

## Create or get the dataset

In [ ]:
# Halcon Anomaly detection datasets
# https://www.mvtec.com/company/research/datasets/mvtec-ad/downloads

In [ ]:
## Download pill dataset
#%%capture
#!wget https://www.mydrive.ch/shares/43421/11a215a5749fcfb75e331ddd5f8e43ee/download/420938129-1629953099/pill.tar.xz
#!tar -xf pill.tar.xz

In [ ]:
## Download toothbrush dataset
#%%capture
#!wget https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938134-1629953256/toothbrush.tar.xz
#!tar -xf toothbrush.tar.xz

In [ ]:
## Download zipper dataset
#%%capture
#!wget https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938385-1629953449/zipper.tar.xz
#!tar -xf zipper.tar.xz

In [ ]:
# Download screw dataset
%%capture
!wget https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938130-1629953152/screw.tar.xz
!tar -xf screw.tar.xz

## Use roboflow to label the dasaset and perform augmentation

In [ ]:
# Roboflow
# https://roboflow.com/

## Use ultralytics hub for training

In [7]:
# Ultralytics hub
# https://www.ultralytics.com/hub

## Test the model using Ultralytics library

In [1]:
%%capture
!pip install ultralytics

In [2]:
from ultralytics import YOLO
import cv2
from PIL import Image
import torch
import numpy as np
import time

In [3]:
class Inf:
    def __init__(self, path_to_model, gpu=False):
        self.device = torch.device("cpu")
        if gpu and torch.cuda.is_available():
            self.device = torch.device("cuda")
        self.model = YOLO(path_to_model).to(self.device)
    def run(self, im_list, conf_thres=0.25, iou_thres=0.45):
        start = time.time()
        res = self.model.predict(im_list, imgsz=640, device=self.device, conf=conf_thres, iou=iou_thres, verbose=False)
        end = time.time()
        print(f"Elapsed: {np.round(end-start,3)}s")
        if not isinstance(im_list, list):
            im_list = [im_list]
        im_draw_list = []
        for i in range(0, len(res)):
            im_draw = im_list[i].copy()
            names = res[i].names
            scores = res[i].boxes.conf.cpu().numpy()
            class_ids = res[i].boxes.cls.cpu().numpy()
            boxes = res[i].boxes.xyxy.cpu().numpy()
            for box, score, class_id in zip(boxes, scores, class_ids):
                x1, y1, x2, y2 = box.astype(int)
                cv2.rectangle(im_draw, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(im_draw, f"{names[class_id]}: {score:.2f}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            im_draw_list.append(im_draw)
        return im_draw_list

In [4]:
%%capture
!wget https://ultralytics.com/images/bus.jpg

In [5]:
net = Inf("yolov8n.pt", gpu=True)
im = cv2.imread("bus.jpg", 1) # BGR # 1=cv2.IMREAD_COLOR, 0=cv2.IMREAD_GRAY
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
print(f"Model is running on: {net.model.device}")
for i in range(0,20):
    #im_draw_list = net.run([im,im,im,im])
    im_draw_list = net.run(im)

#for i in range(0, len(im_draw_list)):
#    display(Image.fromarray(im_draw_list[i]))

100%|██████████| 6.23M/6.23M [00:00<00:00, 138MB/s]


Model is running on: cuda:0
Elapsed: 2.199s
Elapsed: 0.012s
Elapsed: 0.012s
Elapsed: 0.01s
Elapsed: 0.01s
Elapsed: 0.011s
Elapsed: 0.01s
Elapsed: 0.01s
Elapsed: 0.01s
Elapsed: 0.01s
Elapsed: 0.013s
Elapsed: 0.01s
Elapsed: 0.01s
Elapsed: 0.01s
Elapsed: 0.01s
Elapsed: 0.01s
Elapsed: 0.01s
Elapsed: 0.013s
Elapsed: 0.016s
Elapsed: 0.018s


## Export

In [19]:
%%capture
# Ultralytics requirements
!pip install ultralytics

In [20]:
%%capture
# Onnx export requirements
!pip install onnx onnxruntime onnxruntime-gpu

In [ ]:
# Export the model in onnx format
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
model.export(format='onnx')

In [22]:
%%capture
# Openvino requirements
!pip install openvino-dev

In [ ]:
# Export the model in openvino format
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
model.export(format='openvino')  # creates 'yolov8n_openvino_model/'

## Test

In [ ]:
%%capture
!wget https://ultralytics.com/images/bus.jpg

In [13]:
from ultralytics import YOLO
import cv2
from PIL import Image
import torch
import numpy as np
import time

In [15]:
def get_time_measure(model, im_list, num_it=10):
    if not isinstance(im_list, list):
        im_list = [im_list]
    time_el = []
    for i in range(0,num_it):
        start = time.time()
        res = model.predict(im_list, imgsz=640, conf=0.25, iou=0.45, verbose=False)
        end = time.time()

        im_draw_list = []
        for i in range(0, len(res)):
            im_draw = im_list[i].copy()
            names = res[i].names
            scores = res[i].boxes.conf.cpu().numpy()
            class_ids = res[i].boxes.cls.cpu().numpy()
            boxes = res[i].boxes.xyxy.cpu().numpy()
            for box, score, class_id in zip(boxes, scores, class_ids):
                x1, y1, x2, y2 = box.astype(int)
                cv2.rectangle(im_draw, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(im_draw, f"{names[class_id]}: {score:.2f}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            im_draw_list.append(im_draw)
        time_el.append(end-start)
    time_res = np.round(np.median(time_el),5)
    return time_res

In [16]:
# read Image
im = cv2.imread("bus.jpg", 1) # BGR # 1=cv2.IMREAD_COLOR, 0=cv2.IMREAD_GRAY
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

In [17]:
# CPU
model_cpu_pytorch = YOLO("yolov8n.pt", task="detect")
device = torch.device("cpu")
model_cpu_pytorch = model_cpu_pytorch.to(device)
time_cpu_pytorch = get_time_measure(model_cpu_pytorch, im)
print(f"Pytorch cpu: {time_cpu_pytorch}s")

Pytorch cpu: 0.13198s


In [32]:
# GPU
if torch.cuda.is_available():
    model_gpu_pytorch = YOLO("yolov8n.pt", task="detect")
    device = torch.device("cuda")
    model_gpu_pytorch = model_gpu_pytorch.to(device)
    print("Model running on gpu")
    time_gpu_pytorch = get_time_measure(model_gpu_pytorch, im)
    print(f"Pytorch gpu: {time_gpu_pytorch}s")
else:
    print("No gpu available")

Model running on gpu
Pytorch gpu: 0.01042s


In [26]:
# Onnx CPU
model_cpu_onnx = YOLO("yolov8n.onnx", task="detect")
time_cpu_onnx = get_time_measure(model_cpu_onnx, im)
print(f"Onnx cpu: {time_cpu_onnx}s")

Loading yolov8n.onnx for ONNX Runtime inference...
Onnx cpu: 0.13426s


In [31]:
# Openvino CPU
model_cpu_openvino = YOLO("yolov8n_openvino_model/", task="detect")
time_cpu_openvino = get_time_measure(model_cpu_openvino, im)
print(f"Openvino cpu: {time_cpu_openvino}s")

Loading yolov8n_openvino_model for OpenVINO inference...
Openvino cpu: 0.09912s
